In [9]:
from pathlib import Path
import re

In [10]:
import hashlib
from difflib import unified_diff


# go through all folders and files in ./comparison/old and for each file
# find the corresponding file in ./comparison/new and compare the two files
def compare_dirs(dir_one: Path, dir_two: Path, print_diff: bool = False):
    for old_file in old_dir.rglob("**/*"):
        if old_file.suffix in [".txt", ".xml"]:
            new_file = new_dir / old_file.relative_to(old_dir)
            if new_file.exists():
                with open(old_file, "r") as f:
                    old_content = f.read()
                with open(new_file, "r") as f:
                    new_content = f.read()

                # Calculate the hash of the old content
                old_hash = hashlib.md5(old_content.encode()).hexdigest()

                # Calculate the hash of the new content
                new_hash = hashlib.md5(new_content.encode()).hexdigest()

                # Compare the hashes
                if old_hash != new_hash:
                    print(f"Files {old_file} and {new_file} differ.")

                    # show difference with difflib
                    if print_diff:
                        for line in unified_diff(
                            old_content.splitlines(),
                            new_content.splitlines(),
                            lineterm="",
                        ):
                            print(line)

            else:
                print(f"File {new_file} does not exist.")

In [12]:
old_dir = Path("../comparison/old")
new_dir = Path("../comparison/new")

compare_dirs(old_dir, new_dir, print_diff=True)

Files ../comparison/old/electoral_term_pp02/02190/session_content.txt and ../comparison/new/electoral_term_pp02/02190/session_content.txt differ.
--- 
+++ 
@@ -290,7 +290,8 @@
 wie man es stärker überhaupt nicht unterstreichen kann.
 (Beifall bei der FDP, bei der SPD und beim GB/BHE.)
 Denn wenn der Herr Bundeskanzler recht behält, meine Damen und Herren, dann müssen wir uns doch geradezu beeilen, alles zu tun, um nicht etwa überrascht zu werden.
-(Beifall bei der FDP, bei der SPD und beim GB/BHE. - Zuruf von der SPD: Also doch Wahlschlager!)
+(Beifall bei der FDP, bei der SPD und beim GB/BHE. - Zuruf von der SPD:
+Also doch Wahlschlager!)
 Und ein Zweites, meine Damen und Herren. Der Herr Bundesminister des Innern hat hier etwas gesagt, was ich wieder sehr bedauert habe. Wir haben frühere Berlin-Debatten hier ohne parteipolitische Aspekte geführt. Das ist langsam anders geworden. Ich möchte diesen Bundestag nicht kränken, aber im großen und ganzen war es im 1. Bundestag in dieser Bezi

In [1]:
# Deletes all the newlines in brackets
import regex


def f1(filetext):
    bracket_text = regex.finditer(r"\(([^(\)]*(\(([^(\)]*)\))*[^(\)]*)\)", filetext)

    for bracket in bracket_text:
        filetext = filetext.replace(
            str(bracket.group()),
            regex.sub(
                r"\n+",
                " ",
                regex.sub(
                    r"(^((?<!Abg\.).)+|^.*\[.+)(-\n+)",
                    r"\1",
                    str(bracket.group()),
                    flags=regex.MULTILINE,
                ),
            ),
        )

    return filetext


def f2(filetext: str) -> str:
    # Finds all text within parentheses, including nested parentheses
    bracket_text = regex.findall(r"\(([^(\)]*(\(([^(\)]*)\))*[^(\)]*)\)", filetext)

    for sub_groups in bracket_text:
        # findall does not return the outer brackets compared to finditer
        bracket_content = sub_groups[0]
        bracket_content = regex.sub(
            r"(^((?<!Abg\.).)+|^.*\[.+)(-\n+)",
            r"\1",
            bracket_content,
            flags=regex.MULTILINE,
        )
        bracket_content = regex.sub(r"\n+", " ", bracket_content)
        filetext = filetext.replace(sub_groups[0], bracket_content)

    return filetext

In [19]:
text = """(Hört! Hört! bei der SPD und auf der
äußersten Rechten. — Zuruf des Abg. Dr.
Bucerius: Er erkennt also unsere Ansprüche an!)"""

print(f1(text))


(Hört! Hört! bei der SPD und auf der äußersten Rechten. — Zuruf des Abg. Dr. Bucerius: Er erkennt also unsere Ansprüche an!)


In [20]:
print(f2(text))

(Hört! Hört! bei der SPD und auf der äußersten Rechten. — Zuruf des Abg. Dr. Bucerius: Er erkennt also unsere Ansprüche an!)


In [21]:
f1(text) == f2(text)

True